In [8]:
import cvzone
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector

In [ ]:
cap = cv2.VideoCapture(0)

cap.set(3,1280)
cap.set(4,720)

Background = cv2.imread(r"D:\Me\Ai\Computer vision projects\bing pong Game\MOMEN pong (1)\1.jpg")
Gameover = cv2.imread(r"D:\Me\Ai\Computer vision projects\bing pong Game\MOMEN pong (1)\2.jpg")
Ball = cv2.imread(r"D:\Me\Ai\Computer vision projects\bing pong Game\MOMEN pong (1)\Ball.png", cv2.IMREAD_UNCHANGED)
Bat1 = cv2.imread(r"D:\Me\Ai\Computer vision projects\bing pong Game\MOMEN pong (1)\1-removebg-preview.png", cv2.IMREAD_UNCHANGED)
Bat2 = cv2.imread(r"D:\Me\Ai\Computer vision projects\bing pong Game\MOMEN pong (1)\2-removebg-preview.png", cv2.IMREAD_UNCHANGED)


# Hand Detector
detector = HandDetector(detectionCon=0.8, maxHands=2)

# Variables
ballPos = [100, 100]
speedX = 15
speedY = 15
gameOver = False
score = [0, 0]

while True:
    _, img = cap.read()
    img = cv2.flip(img, 1)
    imgRaw = img.copy()

    # Find the hand and its landmarks
    hands, img = detector.findHands(img, flipType=False)  # with draw

    # Overlaying the background image
    img = cv2.addWeighted(img, 0.2, Background, 0.8, 0)

    # Check for hands
    if hands:
        for hand in hands:
            x, y, w, h = hand['bbox']
            h1, w1, _ = Bat1.shape
            y1 = y - h1 // 2
            y1 = np.clip(y1, 20, 415)

            if hand['type'] == "Left":
                img = cvzone.overlayPNG(img, Bat1, (0, y1))
                if -50 < ballPos[0] < w1-50 and y1 < ballPos[1] < y1 + h1:
                    speedX = -speedX
                    ballPos[0] += 30
                    score[0] += 1

            if hand['type'] == "Right":
                img = cvzone.overlayPNG(img, Bat2, (1120, y1))
                if 1195 - 50  < ballPos[0] < 1195 and y1 < ballPos[1] < y1 + h1:
                    speedX = -speedX
                    ballPos[0] -= 30
                    score[1] += 1

    # Game Over
    if ballPos[0] < 40 or ballPos[0] > 1200:
        gameOver = True

    if gameOver:
        img = Gameover
        cv2.putText(img, str(score[1] + score[0]).zfill(2), (570, 440), cv2.FONT_HERSHEY_COMPLEX,
                    2.5, (200, 0, 200), 5)

    # If game not over move the ball
    else:

        # Move the Ball
        if ballPos[1] >= 500 or ballPos[1] <= 10:
            speedY = -speedY

        ballPos[0] += speedX
        ballPos[1] += speedY

        # Draw the ball
        img = cvzone.overlayPNG(img, Ball, ballPos)

        cv2.putText(img, str(score[0]), (300, 650), cv2.FONT_HERSHEY_COMPLEX, 3, (255, 255, 255), 5)
        cv2.putText(img, str(score[1]), (900, 650), cv2.FONT_HERSHEY_COMPLEX, 3, (255, 255, 255), 5)

    img[580:700, 20:233] = cv2.resize(imgRaw, (213, 120))

    cv2.imshow("Image", img)
    key = cv2.waitKey(1)
    if key == ord('r'):
        ballPos = [100, 100]
        speedX = 5
        speedY = 5
        gameOver = False
        score = [0, 0]
        Gameover = cv2.imread(r"D:\Me\Ai\Computer vision projects\bing pong Game\MOMEN pong (1)\2.jpg")
    elif cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()